In [12]:
import cudaq
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Function
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [13]:
cudaq.set_target("nvidia")

In [14]:
def ry(theta, qubit):
    cudaq.ry(theta, qubit)

def rx(theta, qubit):
    cudaq.rx(theta, qubit)

In [16]:
# Define quantum function using CUDA-Q
class QuantumFunction(Function):
    def __init__(self, qubit_count: int, hamiltonian: cudaq.SpinOperator):
        @cudaq.kernel
        def kernel(qubit_count: int, thetas: np.ndarray):
            qubits = cudaq.qvector(qubit_count)
            ry(thetas[0], qubits[0])
            rx(thetas[1], qubits[0])
        self.kernel = kernel
        self.qubit_count = qubit_count
        self.hamiltonian = hamiltonian

    def run(self, theta_vals: torch.Tensor) -> torch.Tensor:
        theta_vals_np = theta_vals.cpu().numpy()
        qubit_count = [self.qubit_count] * theta_vals_np.shape[0]
        results = cudaq.observe(self.kernel, self.hamiltonian, qubit_count, theta_vals_np)
        return torch.tensor([res.expectation() for res in results], dtype=torch.float32)

    @staticmethod
    def forward(ctx, thetas: torch.Tensor, quantum_circuit, shift) -> torch.Tensor:
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit
        exp_vals = ctx.quantum_circuit.run(thetas).reshape(-1, 1)
        ctx.save_for_backward(thetas, exp_vals)
        return exp_vals

    @staticmethod
    def backward(ctx, grad_output):
        thetas, _ = ctx.saved_tensors
        gradients = torch.zeros_like(thetas)
        for i in range(thetas.shape[1]):
            thetas_plus = thetas.clone()
            thetas_plus[:, i] += ctx.shift
            thetas_minus = thetas.clone()
            thetas_minus[:, i] -= ctx.shift
            exp_plus = ctx.quantum_circuit.run(thetas_plus)
            exp_minus = ctx.quantum_circuit.run(thetas_minus)
            gradients[:, i] = (exp_plus - exp_minus) / (2 * ctx.shift)
        return gradients * grad_output, None, None

In [17]:
# Quantum layer
class QuantumLayer(nn.Module):
    def __init__(self, qubit_count: int, hamiltonian, shift: torch.Tensor):
        super().__init__()
        self.quantum_circuit = QuantumFunction(qubit_count, hamiltonian)
        self.shift = shift

    def forward(self, input: torch.Tensor):
        return QuantumFunction.apply(input, self.quantum_circuit, self.shift)

In [18]:
# Hybrid QNN model
class Hybrid_QNN(nn.Module):
    def __init__(self, input_dim=8):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.25)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 2)
        self.quantum = QuantumLayer(qubit_count=2, hamiltonian=cudaq.spin.z(0), shift=torch.tensor(np.pi / 2))

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.quantum(x)
        return x

In [21]:
df = pd.read_csv("../data/noalpha.csv")
df.head()
df.columns

Index(['Unnamed: 0', 'ID', 'RAdeg', 'DEdeg', 'e_RAdeg', 'e_DEdeg', 'RApeak',
       'DEpeak', 'Sint', 'e_Sint', 'Speak', 'e_Speak', 'rmspeak', 'e_rmspeak',
       'thetamaj', 'thetamin', 'PA'],
      dtype='object')

In [22]:
features = df.drop(columns=['thetamaj', 'thetamin','ID','Unnamed: 0'])
target_ma = df['thetamaj'].values.reshape(-1, 1)
target_mi = df['thetamin'].values.reshape(-1, 1)

In [26]:
# Scale features and targets
scaler_X = MinMaxScaler()
scaler_y_ma = MinMaxScaler()
scaler_y_mi = MinMaxScaler()

X_scaled = scaler_X.fit_transform(features)
y_ma_scaled = scaler_y_ma.fit_transform(target_ma).flatten()
y_mi_scaled = scaler_y_mi.fit_transform(target_mi).flatten()

X_train, X_test, y_train_ma, y_test_ma = train_test_split(X_scaled, y_ma_scaled, test_size=0.2, random_state=42)
_, _, y_train_mi, y_test_mi = train_test_split(X_scaled, y_mi_scaled, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_ma_tensor = torch.tensor(y_train_ma, dtype=torch.float32)
y_test_ma_tensor = torch.tensor(y_test_ma, dtype=torch.float32)
y_train_mi_tensor = torch.tensor(y_train_mi, dtype=torch.float32)
y_test_mi_tensor = torch.tensor(y_test_mi, dtype=torch.float32)


In [27]:
# Train function
def train_model(model, X_train, y_train, epochs=50, lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train).flatten()
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.6f}")

In [28]:
model_ma = Hybrid_QNN(input_dim=X_train.shape[1])
train_model(model_ma, X_train_tensor, y_train_ma_tensor)
model_ma.eval()
with torch.no_grad():
    y_pred_ma = model_ma(X_test_tensor).cpu().numpy().flatten()


Epoch 1/50, Loss: 0.807598
Epoch 2/50, Loss: 0.798762
Epoch 3/50, Loss: 0.787685
Epoch 4/50, Loss: 0.773179
Epoch 5/50, Loss: 0.754731
Epoch 6/50, Loss: 0.731046
Epoch 7/50, Loss: 0.700728
Epoch 8/50, Loss: 0.661586
Epoch 9/50, Loss: 0.611638
Epoch 10/50, Loss: 0.547862
Epoch 11/50, Loss: 0.469849
Epoch 12/50, Loss: 0.378831
Epoch 13/50, Loss: 0.277388
Epoch 14/50, Loss: 0.177474
Epoch 15/50, Loss: 0.092216
Epoch 16/50, Loss: 0.034869
Epoch 17/50, Loss: 0.011978
Epoch 18/50, Loss: 0.009944
Epoch 19/50, Loss: 0.010963
Epoch 20/50, Loss: 0.013482
Epoch 21/50, Loss: 0.030384
Epoch 22/50, Loss: 0.058046
Epoch 23/50, Loss: 0.073446
Epoch 24/50, Loss: 0.066947
Epoch 25/50, Loss: 0.047648
Epoch 26/50, Loss: 0.029094
Epoch 27/50, Loss: 0.020258
Epoch 28/50, Loss: 0.018623
Epoch 29/50, Loss: 0.020944
Epoch 30/50, Loss: 0.023913
Epoch 31/50, Loss: 0.025931
Epoch 32/50, Loss: 0.026461
Epoch 33/50, Loss: 0.025176
Epoch 34/50, Loss: 0.023582
Epoch 35/50, Loss: 0.021441
Epoch 36/50, Loss: 0.018638
E

In [29]:
print("\n--- Thetamaj Metrics ---")
print("R2:", r2_score(y_test_ma_tensor, y_pred_ma))
print("MAE:", mean_absolute_error(y_test_ma_tensor, y_pred_ma))
print("RMSE:", np.sqrt(mean_squared_error(y_test_ma_tensor, y_pred_ma)))


--- Thetamaj Metrics ---
R2: -0.7333278656005859
MAE: 0.06106337159872055
RMSE: 0.08946573510055113


In [30]:
model_mi = Hybrid_QNN(input_dim=X_train.shape[1])
train_model(model_mi, X_train_tensor, y_train_mi_tensor)
model_mi.eval()
with torch.no_grad():
    y_pred_mi = model_mi(X_test_tensor).cpu().numpy().flatten()

print("\n--- Thetamin Metrics ---")
print("R2:", r2_score(y_test_mi_tensor, y_pred_mi))
print("MAE:", mean_absolute_error(y_test_mi_tensor, y_pred_mi))
print("RMSE:", np.sqrt(mean_squared_error(y_test_mi_tensor, y_pred_mi)))

Epoch 1/50, Loss: 0.672833
Epoch 2/50, Loss: 0.671109
Epoch 3/50, Loss: 0.668764
Epoch 4/50, Loss: 0.665352
Epoch 5/50, Loss: 0.660326
Epoch 6/50, Loss: 0.653236
Epoch 7/50, Loss: 0.643732
Epoch 8/50, Loss: 0.630749
Epoch 9/50, Loss: 0.613111
Epoch 10/50, Loss: 0.590126
Epoch 11/50, Loss: 0.559494
Epoch 12/50, Loss: 0.519679
Epoch 13/50, Loss: 0.468061
Epoch 14/50, Loss: 0.403064
Epoch 15/50, Loss: 0.325401
Epoch 16/50, Loss: 0.236109
Epoch 17/50, Loss: 0.147110
Epoch 18/50, Loss: 0.073611
Epoch 19/50, Loss: 0.047446
Epoch 20/50, Loss: 0.088637
Epoch 21/50, Loss: 0.155979
Epoch 22/50, Loss: 0.174367
Epoch 23/50, Loss: 0.150303
Epoch 24/50, Loss: 0.105869
Epoch 25/50, Loss: 0.067706
Epoch 26/50, Loss: 0.046013
Epoch 27/50, Loss: 0.043807
Epoch 28/50, Loss: 0.053214
Epoch 29/50, Loss: 0.065629
Epoch 30/50, Loss: 0.076858
Epoch 31/50, Loss: 0.082522
Epoch 32/50, Loss: 0.080754
Epoch 33/50, Loss: 0.074716
Epoch 34/50, Loss: 0.063468
Epoch 35/50, Loss: 0.051418
Epoch 36/50, Loss: 0.041387
E